# Link based ranking



## 📚 Exercise 1: Page rank  (Eigen-vector method)

In the first exercise, we implement the Page Rank algorithm using Eigen vector method.


### Goal:
1. Implementing Page rank with Eigen-vector method.
2. Testing the implemented method on a small Web with three pages (with four different link structures).

### What you are learning in this exercise:
1. Analyzing the output of PageRank for different link structures.

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, you can use `numpy` library as follows:

In [2]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


## Links structure
Consider a tiny Web with three pages A, B and C with no inlinks, and with initial PageRank = 1. Initially, none of the pages link to any other pages and none link to them. 
Run your algorithm for the following cases, and calculate the PageRank for
each case:

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

**Hints:**
Please note that for this section, we are using the theoretical PageRank computation (without source of rank). See the slide "Transition Matrix for Random Walker" in the lecture notes. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice.

In [13]:
# Implement your code here
def pagerank_eigen(L, R=None):
    if R is None: # We might want to compute R outside this function to avoid recomputing large matrix
        # Construct transition probability matrix from L
        _,cols = L.shape
        R = np.zeros(L.shape)
        for col in range(cols):
            R[:][col] = L[:][col] / np.linalg.norm(L[:][col],2)
    # Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = np.linalg.eig(R)
    # Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:,np.argmax(eigenvalues)]
    return (R,p)

In [14]:
# Test with a sample case, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.         0.70710678 0.70710678]
 [0.70710678 0.         0.70710678]
 [0.70710678 0.70710678 0.        ]]
p=[0.57735027 0.57735027 0.57735027]


## 📚 Exercise 2: Page rank  (Iterative method)

In the second exercise, we implement the Page Rank algorithm with an iterative approach.


### Goal:
1. Implementing Page rank with the iterative method.
2. Reading the `ca-GrQc.txt` dataset and constructing its link matrix
3. Testing the implemented algorithm on the constructed link matrix
4. comparing the run-time of iterative method and eigen-vector method.

### What you are learning in this exercise:
1. Benefits of the iterative approach over eigen-vector method.

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same paths of this notebook.

In [21]:
def pagerank_iterative(L, R=None):
    if R is None:  # We might want to compute R outside this function to avoid recomputing large matrix
        R = np.multiply(L, 1 / np.sum(L,axis=0))
        _,cols = L.shape
        R = np.zeros(L.shape)
        for col in range(cols):
            R[:][col] = L[:][col] / np.linalg.norm(L[:][col],2)
    N = L.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = q * np.matmul(R,p)
        p = p + (1-q) / N * e
        delta = np.linalg.norm(p - p_prev, 1)
        i += 1

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

### Construct link matrix from dataset

In [22]:
# Construct link matrix from file
n_nodes = 0 # number of nodes in the dataset
nodes_idx = dict() #Since the nodeIDs are not from 0 to N we need to build an index of nodes
nodes = [] #We also want to store nodeIDs to return the result of ranking vector
links = []

with open("ca-GrQc.txt") as f:
    # Fill in your code
    lines = f.readlines()
    for line in lines: 
        line = line.split()
        if line[0] != '#': 
            if line[0] not in nodes_idx: 
                nodes_idx[line[0]] = n_nodes
                nodes.append(line[0])
                n_nodes += 1 
            if line[1] not in nodes_idx: 
                nodes_idx[line[1]] = n_nodes
                n_nodes += 1
                nodes.append(line[1])
            links.append((nodes_idx[line[0]], nodes_idx[line[1]]))
    
L = np.zeros((n_nodes, n_nodes))
for src, dst in links: 
    L[src][dst] = 1

## Comparing the run-time of eigen-vector and iterative methods

### You will see that eigen-vector method is slow and has some numerical issues


#### First we pre-compute the R matrix and pass it as argument to Page rank algorithms.

In [23]:
R = np.multiply(L, 1 / np.sum(L,axis=0))  # note that this doesn't handle zero columns
print(R)

[[0.    0.2   0.5   ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.5   0.5  ]
 [0.    0.    0.    ... 0.5   0.    0.5  ]
 [0.    0.    0.    ... 0.5   0.5   0.   ]]


In [19]:
import time
start_time = time.time()
# Run PageRank
R,p = pagerank_eigen(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p))

--- 60.53269386291504 seconds ---
Ranking vector: p=[-2.29217365e-05+0.j -1.43260853e-05+0.j -5.73043413e-06+0.j ...
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]


### Iterative method

In [24]:
start_time = time.time()
# Run PageRank
R, p = pagerank_iterative(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p[:,0]))

Converged after 128 iterations. Ranking vector: p=[2.91315639e-04 1.88382754e-04 8.39741651e-05 ... 1.92156702e-04
 1.92156702e-04 1.92156702e-04]
--- 0.5931730270385742 seconds ---
Ranking vector: p=[2.91315639e-04 1.88382754e-04 8.39741651e-05 ... 1.92156702e-04
 1.92156702e-04 1.92156702e-04]


## Now let's extract top-k nodes from the ranking vector

In [26]:
arr = np.array(p[:,0])
k = 3
k_idx = np.argpartition(p.ravel(),-3)[-3:] # index of top-k nodes
print("Top-{0} nodes: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

Top-3 nodes: ['13929' '14265' '13801']
Their scores: [0.00138011 0.00144951 0.00141553]


## 📚 Exercise 3: Ranking Methodology  (Hard)
This exercise is theoretical and you don't need to write code for it:

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

## 📚 Exercise 4: Hub and Authority

In the last exercise, we implement the (iterative) HITS algorithm to calculate the authority and hub scores for this graph.


### Goal:
1. Implementing HITS algorithm.
2. Identifying the best authority and hub nodes for a small graph (part `a`) as well as the `ca-GrQc.txt` dataset (part `b`)


### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes. (A non-zero element $A_{ij}$ indicates an edge from $i$ to $j$.)

### b)
Apply the HITS algorithm to the `ca-GrQc.txt` dataset (the same dataset as for the `Exercise 2`)

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [35]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]
    x0, y0 = 1 / (N**0.5) * np.ones(N), 1 / (N**0.5) * np.ones(N)
    xprev, yprev = x0, y0
    
    # For advanced exercise: define a convergence condition instead of k iterations
    for l in range(0,k):
        y = (np.matmul((A),xprev))
        x = (np.matmul(np.transpose(A),yprev))
        xprev = x / np.linalg.norm(x, 2)
        yprev = y / np.linalg.norm(y, 2)
        
    return xprev, yprev

### Part `a` dataset

In [36]:
A= np.array([[0,1,1,1], [0,0,1,1], [1,0,0,1], [0,0,0,1]]) # your code here

x, y = hits_iterative(A, k=6)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Result using iterative method:
 Authoriy vector x=[0.16887796 0.27257494 0.49774555 0.80587375]
 Hub vector y=[0.65357971 0.54153747 0.40815386 0.33612671]


### part `b` dataset
You can reuse the link matrix $L$ (from Exercise 2) to compute the adjacency matrix $A$ of the dataset

In [37]:
A = L # your code here
print(A)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


Run HITS algorithm:

In [38]:
x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Result using iterative method:
 Authoriy vector x=[3.48948725e-005 1.45965100e-005 6.68130767e-006 ... 1.83796992e-137
 1.83796992e-137 1.83796992e-137]
 Hub vector y=[3.48948725e-005 1.45965100e-005 6.68130767e-006 ... 1.83796992e-137
 1.83796992e-137 1.83796992e-137]
